In [1]:
from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np
import autograd.numpy.random as npr
import autograd.scipy as asp
import scipy as sp

import copy
from scipy import optimize

In [2]:
# Simulate data from a linear model y_vec = x_mat * beta + Gaussian noise

N = 200     # observations per group
K = 5      # dimension of regressors

# Generate data
true_beta = np.random.random(K)
true_beta = true_beta - np.mean(true_beta)
true_y_info = 1.0

x_mat = np.random.random(K * N).reshape(N, K) - 0.5
true_mean = np.matmul(x_mat, true_beta)
y_vec = np.random.normal(true_mean, 1 / np.sqrt(true_y_info), N)

In [21]:
# Store all the relevant data in a class.

class LeastSquaresObjective(object):
    def __init__(self, x_mat, y_vec):
        self.x_mat = copy.deepcopy(x_mat)
        self.y_vec = copy.deepcopy(y_vec)
        self.weights = np.full(x_mat.shape[0], 1.0)
        self.objective_grad = grad(self.objective)
        self.objective_hess = hessian(self.objective)
        self.objective_hvp = hessian_vector_product(self.objective)
        
    def objective(self, beta):
        # Of course, this could be made much faster by caching x^T x and Y^T x.
        return np.sum(self.weights * (y_vec - np.matmul(x_mat, beta))**2)
        
    def weighted_objective(self, beta, weights):
        self.weights = weights
        return self.objective(beta)

ls = LeastSquaresObjective(x_mat, y_vec)

In [22]:
# Optimize the objective using only Hessian vector products via the conugate gradient trust region algorithm.

print 'Running Newton Trust Region'
beta_opt_tr = optimize.minimize(
    ls.objective, np.full(K, 0.0), method='trust-ncg',
    jac=ls.objective_grad, hessp=ls.objective_hvp,
    tol=1e-6, options={'maxiter': 100, 'disp': True, 'gtol': 1e-6 })

Running Newton Trust Region
Optimization terminated successfully.
         Current function value: 223.000023
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
         Hessian evaluations: 0


In [23]:
# We know the optimum in closed form in this case.
beta_opt = np.linalg.solve(np.matmul(x_mat.transpose(), x_mat), np.matmul(x_mat.transpose(), y_vec))

# The known optimum should be the same as the one found by the conjugate gradient trust region method.
print beta_opt
print beta_opt_tr.x

[-0.65076139  0.03547502  0.26804489  0.60808264 -0.91844248]
[-0.65076139  0.03547502  0.26804489  0.60808264 -0.91844248]


In [32]:
# Get the sensitivity to the weights by differentiating the first order condition.
id_weights = np.full(N, 1.0)
weighted_objective_grad = grad(ls.weighted_objective)

# Sanity check: these should be the same.
print np.max(np.abs(weighted_objective_grad(beta_opt, id_weights) - ls.objective_grad(beta_opt)))

# Get the second derivative d2 objective / dbeta dweights, called h_bw
dobj_dbeta_dweight = jacobian(weighted_objective_grad, argnum=1)
h_bw = dobj_dbeta_dweight(beta_opt, id_weights)

# The hessian with respect to beta.
h_bb = ls.objective_hess(beta_opt)

# This is the sensitivity of beta to changing each weight.
w_sensitivity = -1 * np.linalg.solve(h_bb, h_bw)
print w_sensitivity.shape

0.0
(5, 200)


In [27]:
# Re-fit the model with a datapoint removed.

remove_row = 0
new_weights = np.full(N, 1.)
new_weights[remove_row] = 0.
ls.weights = copy.deepcopy(new_weights)

beta_opt_perturb = optimize.minimize(
    ls.objective, np.full(K, 0.0), method='trust-ncg',
    jac=ls.objective_grad, hessp=ls.objective_hvp,
    tol=1e-6, options={'maxiter': 100, 'disp': True, 'gtol': 1e-6 })

Optimization terminated successfully.
         Current function value: 220.981716
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
         Hessian evaluations: 0


In [31]:
# These should be roughly the same, indicating that the sensitivity matrix
# is a good estimator of the effect of removing a data point.

print beta_opt_tr.x - beta_opt_perturb.x
print w_sensitivity[:, remove_row]

[-0.00490503 -0.04509636 -0.04351522  0.03124869  0.00784744]
[-0.00474112 -0.04358936 -0.04206106  0.03020444  0.0075852 ]
